In [1]:
import torch
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime


# Custom Libraries
from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.helper import to_python_native, gen_experiment_name, set_seed, save_model_state
from utils.plotter import alignment_progress_over_iterations, plot_initial_final_accuracies
from utils.qiskit.model import Qkernel
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.test.mock import FakeLima
import numpy as np
import torch

2025-08-04 16:49:05,028	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-08-04 16:49:05,279	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/tmp/ipykernel_69833/442881082.py:23: DeprecationWarning: The module 'qiskit.test.mock' is deprecated since Qiskit Terra 0.21.0, and will be removed 3 months or more later. Instead, you should import the desired object directly 'qiskit.providers.fake_provider'.
  from qiskit.test.mock import FakeLima


In [2]:
# Setup fake backend with noise model
fake_backend = FakeLima()
noise_model = NoiseModel.from_backend(fake_backend)
simulator = AerSimulator(noise_model=noise_model,
                         basis_gates=noise_model.basis_gates,
                         coupling_map=fake_backend.configuration().coupling_map)

In [3]:
data_generator = DataGenerator(
        dataset_name= 'checkerboard',
        file_path='../data/checkerboard_dataset.npy',
    )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset()
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)
multi_class = True if int(training_labels.unique().numel()) == 2 else False


kernel = Qkernel(
    n_qubits = 2,
    trainable = True,
    input_scaling = True,
    data_reuploading = True,
    ansatz = 'embedding_paper',
    ansatz_layers = 3,
    use_noisy_backend=False,
    simulator=None
)

kernel_noisy = Qkernel(
    n_qubits = 2,
    trainable = True,
    input_scaling = True,
    data_reuploading = True,
    ansatz = 'embedding_paper',
    ansatz_layers = 3,
    use_noisy_backend=True,
    simulator=simulator
)

In [4]:
x1 = training_data[0]
x2 = training_data[1]

In [5]:
kernel.get_depth(x1, x2)

30

In [6]:
def kernel_alignment(K1: torch.Tensor, K2: torch.Tensor) -> float:
    K1 = K1.float()
    K2 = K2.float()

    numerator = torch.sum(K1 * K2)
    denominator = torch.norm(K1, p='fro') * torch.norm(K2, p='fro')

    return (numerator / denominator).item()

In [7]:
def generate_kernel_matrix(x1, x2, k):
    n1 = x1.shape[0]
    n2 = x2.shape[0]
    kernel_matrix = torch.zeros((n1, n2), dtype=torch.float32)

    for i in range(n1):
        for j in range(n2):
            kernel_matrix[i, j] = k.forward(x1[i], x2[j])

    return kernel_matrix

In [8]:
km = generate_kernel_matrix(training_data, training_data, kernel)

/home/digvijay/dev/CCKA-Clustered_Centroid_Kernel_Alignment/utils/qiskit/model.py:133: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(kernel_value, dtype=torch.float32)


In [9]:
km_noisy = generate_kernel_matrix(training_data, training_data, kernel_noisy)

In [10]:
alignment_score = kernel_alignment(km, km_noisy)
print(f"Kernel Alignment (Uncentered): {alignment_score:.4f}")

Kernel Alignment (Uncentered): 0.8685


In [13]:
from utils.mitigation import Mitigation

x1 = training_data[0]
x2 = training_data[1]
mitigator = Mitigation(method='ML-GLOBAL',ml_model= 'POLY_REG', degree=2, alpha=1e-3, circuit_depth = kernel.get_depth(x1, x2), n_qubits=2)
diag_vals = torch.diag(km_noisy).cpu().numpy()
mitigator.train_global_decay(diag_vals)
mitigator.get_survival_probability(training_data)

[INFO] ML-GLOBAL model trained with depth and qubits features.
Survival probability λ_i: [0.91224265 0.91219171 0.91214156 0.91209222 0.91204367 0.91199591
 0.91194895 0.91190279 0.91185742 0.91181285 0.91176908 0.9117261
 0.91168392 0.91164254 0.91160195 0.91156216 0.91152316 0.91148497
 0.91144756 0.91141096 0.91137515 0.91134013 0.91130592 0.9112725
 0.91123987 0.91120804 0.91117701 0.91114678 0.91111734 0.91108869]


In [14]:
mitigator.get_survival_probability(diag_vals)

Survival probability λ_i: [0.91224265 0.91219171 0.91214156 0.91209222 0.91204367 0.91199591
 0.91194895 0.91190279 0.91185742 0.91181285 0.91176908 0.9117261
 0.91168392 0.91164254 0.91160195 0.91156216 0.91152316 0.91148497
 0.91144756 0.91141096 0.91137515 0.91134013 0.91130592 0.9112725
 0.91123987 0.91120804 0.91117701 0.91114678 0.91111734 0.91108869]


In [15]:
km_mitigated = mitigator.mitigate(km_noisy)

In [16]:
alignment_score = kernel_alignment(km, km_mitigated)
print(f"Kernel Alignment (Uncentered): {alignment_score:.4f}")

Kernel Alignment (Uncentered): 0.8685
